# Instruction

**Run each cell one by one**

#Load_data

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pprint, pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [2]:
id_1_train="1GFFtOkPJskSjhA7-Klfj33fF5eVWa5sE"
id_1_evaluation="1G7cd67MlLEBn-OoY612T08yBoRoGyXyd"


id_2_train="1AP5V-adbxm-bdOuB-7EtgegqzqtlldxZ"
id_2_evaluation="1-XoFS7lPF9DQSO4ItK_KtorUaLFmA4Pn"

id_3_train="1RFuUrqJPqBWycC_j1b2Vy1DgBr1yQbPG"
id_3_evaluation="1fPSYHnTi5ORcI7QgXjIduAYOYZsoE9tI"

id_4_train="1jovz85ALrSRBZgLNUeeoihSJkEYJT3ZH"
id_4_evaluation="1e6C3vn7sPV-LO-Q1EbLTHvlcT7oqhJ99"


id_5_train="17wrRD332sGeRob6W36MNkQmj9Kc6u5lG"
id_5_evaluation="1JUc3flvygXmhdTL-kM_9pvh25pKfpvNy"

id_6_train="1vROntA0I75Gq85VXa9jwukl-IXxEOCCb"
id_6_evaluation="1pYnce5N2KxxNmFM2sgfxldLpaKgclEiY"

id_7_train="1_qvjVphyrE311iInuVDhhWWWN3BC4-sm"
id_7_evaluation="18V-Tp3Kny9aYz8rxSFsRQrYDtstVF75d"


id_8_train="1Cpi6yNpnMX5oCClCClv7Lcgg2RvxspUF"
id_8_evaluation="13cz0_VMYI0-u2LlEhmy61mGuzmjbmP1B"

id_9_train="1o0gyOZrYsyw68GFhpKFuwoFRn4pVwb7Q"
id_9_evaluation="1KW0_TzquGXygyBksatKEvevm6NYwddEA"



def load(id_train,id_evaluation,subject):
  downloaded = drive.CreateFile({'id':id_train}) 
  downloaded.GetContentFile('A0'+str(subject)+'T.mat')
  downloaded = drive.CreateFile({'id':id_evaluation}) 
  downloaded.GetContentFile('A0'+str(subject)+'E.mat')

load(id_1_train,id_1_evaluation,1)
load(id_2_train,id_2_evaluation,2)
load(id_3_train,id_3_evaluation,3)

load(id_4_train,id_4_evaluation,4)
load(id_5_train,id_5_evaluation,5)
load(id_6_train,id_6_evaluation,6)

load(id_7_train,id_7_evaluation,7)
load(id_8_train,id_8_evaluation,8)
load(id_9_train,id_9_evaluation,9)



# Read Data

In [3]:
from keras.models import model_from_json
import numpy as np
import scipy.io as sio
import glob as glob
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
import keras.backend as K
from tensorflow.keras.layers import Layer
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Activation, add, Dense, Flatten, Dropout, Multiply, Embedding, Lambda
from keras.layers import Conv2D, MaxPooling2D,PReLU,concatenate
from keras import backend as K
import numpy as np
import sys
from keras.callbacks import *
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.losses import categorical_crossentropy
import numpy as np
from keras.models import Model
from keras.layers.core import Dense, Activation
from keras.layers.convolutional import Conv1D,Conv2D, AveragePooling2D,SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dropout, Add, Lambda,DepthwiseConv2D,Input, Permute,Flatten
from tensorflow.keras.constraints import max_norm
import random
import keras
from sklearn.decomposition import PCA

def load_all_data (crossValidation, data_path): 

    big_X_train, big_y_train, big_X_test, big_y_test = [None]*9, [None]*9, [None]*9, [None]*9
    for subject in range (0,9):
        path = data_path+'s' + str(subject+1) + '/'
        big_X_train[subject], big_y_train[subject] = get_data(subject+1, True ,path)
        big_X_test[subject], big_y_test[subject] = get_data(subject+1, False ,path)
    
    return big_X_train, big_y_train, big_X_test, big_y_test

def get_data(subject,training,path, highpass = False):
	'''	Loads the dataset 2a of the BCI Competition IV
	available on http://bnci-horizon-2020.eu/database/data-sets

	Keyword arguments:
	subject -- number of subject in [1, .. ,9]
	training -- if True, load training data
				if False, load testing data
	
	Return:	data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
			class_return 	numpy matrix 	size = NO_valid_trial
	'''
	NO_channels = 22
	NO_tests = 6*48 	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(path+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(path+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2= [a_data1[0,0]]
		a_data3= a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]

		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

def prepare_features(path,subject,crossValidation=False):

    fs = 250 
    # t1 = int(1.5*fs)
    t1 = int(2*fs)

    t2 = int(6*fs)
    T = t2-t1
    X_train, y_train = get_data(subject+1,True,path)
    if crossValidation:
        X_train, X_test, y_train, y_test = train_test_split(
            X_train, y_train, test_size=0.2, random_state=0)
    else:
        X_test, y_test = get_data(subject+1,False,path)

    # prepare training data 	
    N_tr,N_ch,_ =X_train.shape 
    X_train = X_train[:,:,t1:t2].reshape(N_tr,1,N_ch,T)
    y_train_onehot = (y_train-1).astype(int)
    y_train_onehot = to_categorical(y_train_onehot)
    # prepare testing data 
    N_test,N_ch,_ =X_test.shape 
    X_test = X_test[:,:,t1:t2].reshape(N_test,1,N_ch,T)
    y_test_onehot = (y_test-1).astype(int)
    y_test_onehot = to_categorical(y_test_onehot)	

    return X_train,y_train,y_train_onehot,X_test,y_test,y_test_onehot

In [4]:
import os
def empty_dir(folder):
    """
    Empty a folder
    :param folder:
    :return:
    """
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            
def build_empty_dir(folder, root_dir=os.getcwd()):
    """
    Build (if required) and empty a directory
    :param folder:
    :param root_dir:
    :return:
    """
    os.makedirs(os.path.join(root_dir, folder), exist_ok=True)
    empty_dir(os.path.join(root_dir, folder))

# Center loss layer

In [5]:
class CenterLossLayer(Layer):

    def __init__(self, alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(4, 120),
                                       initializer='uniform',
                                       trainable=False)
        # self.counter = self.add_weight(name='counter',
        #                                shape=(1,),
        #                                initializer='zeros',
        #                                trainable=False)  # just for debugging
        super().build(input_shape)

    def call(self, x, mask=None):

        # x[0] is Nx2, x[1] is Nx10 onehot, self.centers is 10x2
        delta_centers = K.dot(K.transpose(x[1]), (K.dot(x[1], self.centers) - x[0]))  # 10x2
        center_counts = K.sum(K.transpose(x[1]), axis=1, keepdims=True) + 1  # 10x1
        delta_centers /= center_counts
        new_centers = self.centers - self.alpha * delta_centers
        # self.add_update((self.centers, new_centers), x)
        
        self.centers.assign(new_centers)


        # self.add_update((self.counter, self.counter + 1), x)

        self.result = x[0] - K.dot(x[1], self.centers)
        # self.result = K.sum(self.result ** 2, axis=1, keepdims=True)#/ K.dot(x[1], center_counts)
        self.result = K.mean(self.result ** 2, axis=1, keepdims=True)
        return self.result # Nx1

    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

    def get_config(self):
      cfg = super().get_config()
      return cfg  

### custom loss
def zero_loss(y_true, y_pred):
    # return 0.5 * K.mean(y_pred, axis=0)
    return 0.5 * K.mean(y_pred)



In [6]:
def EEGNet(F1=8,kernLength=32,D=2,Chans=22,dropout=0.2,regRate=.25,averpool_1=8,averpool_2=8):
    input1 = Input(shape = (1,Chans, 1000))
    input2 = Permute((3,2,1))(input1)
    
    block1= Conv2D(F1, (kernLength, 1), padding = 'same',data_format='channels_last',use_bias = False)(input2)
    block1 = BatchNormalization(axis = -1)(block1)

    block2 = DepthwiseConv2D((1, Chans), use_bias = False, 
                                    depth_multiplier = D,
                                    data_format='channels_last',
                                    depthwise_constraint = max_norm(1.))(block1)
    block2 = BatchNormalization(axis = -1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((averpool_1,1),data_format='channels_last')(block2)
    block2 = Dropout(dropout)(block2)

    block3 = SeparableConv2D(F1, (int(kernLength/2), 1),
                            data_format='channels_last',
                            use_bias = False, padding = 'same')(block2)
    block3 = BatchNormalization(axis = -1)(block3)        

    block3 = Activation('elu')(block3)
    block3 = AveragePooling2D((averpool_2,1),data_format='channels_last')(block3)
    block3 = Dropout(dropout)(block3)
    block3=Flatten()(block3)

    
    side = Activation(None, name = 'mmd')(block3)
    aux_input = Input((4,))
    center_side=CenterLossLayer(alpha=0.5, name='centerlosslayer')([block3, aux_input])


    dense = Dense(4, name = 'fe',kernel_constraint = max_norm(regRate))(block3)
    main = Activation('softmax', name = 'softmax')(dense)

    return Model([input1,aux_input], outputs=[main,side,center_side])

In [7]:
model=EEGNet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 22, 1000  0           []                               
                                )]                                                                
                                                                                                  
 permute (Permute)              (None, 1000, 22, 1)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 1000, 22, 8)  256         ['permute[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 1000, 22, 8)  32         ['conv2d[0][0]']             

#MMD construct 

In [8]:
import numpy as np
from matplotlib import pyplot as plt
import math, os
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D, Conv2DTranspose, Dense, Input, Reshape, Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
import pdb
from keras import backend as K
from keras.datasets import mnist
'MMD functions'
def compute_kernel(x, y):
    x_size = K.shape(x)[0]
    y_size = K.shape(y)[0]
    dim = K.shape(x)[1]
    tiled_x = K.tile(K.reshape(x, [x_size, 1, dim]), [1, y_size, 1])
    tiled_y = K.tile(K.reshape(y, [1, y_size, dim]), [x_size, 1, 1])
    return K.exp(-K.mean(K.square(tiled_x - tiled_y), axis=2) / K.cast(dim, 'float32'))

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return K.mean(x_kernel) + K.mean(y_kernel) - 2 * K.mean(xy_kernel)

def custom_loss(source_or_target, y_pred):
    """
    Critical loss builder
		:param train_z: latent code
		:param train_xr: reconstructed data
		:param train_x: training data, the input data
		:return: new loss
		"""
    'So, we first get the mmd loss'
    'First, sample from random noise'
    source_or_target=tf.reshape(source_or_target,[-1])
    TF_source = tf.reshape(tf.where(tf.equal(source_or_target, 0)),[-1])
    TF_target = tf.reshape(tf.where(tf.equal(source_or_target, 1)),[-1])
    Source_fea=tf.gather(y_pred, TF_source)
    # batch_size = K.shape(y_pred)[0]
    # latent_dim = K.int_shape(y_pred)[1]
    # true_samples = K.random_normal(shape=(batch_size, latent_dim), mean=10., stddev=1.)
    Target_fea=tf.gather(y_pred, TF_target)
    
    
    'calculate mmd loss'

    loss_mmd = compute_mmd(Target_fea, Source_fea)

    'Then, also get the reconstructed loss'
    # loss_nll =  = K.categorical_crossentropy(y_true, y_pred)

    'Add them together, then you can get the final loss'
    loss =  loss_mmd*120
    return loss

# Model complie

In [9]:
def model_complie(mdd_weight,center_weight):
  opt = Adam(learning_rate=0.001)
  model=EEGNet()
  model.compile(optimizer=opt, loss=["categorical_crossentropy",custom_loss,zero_loss],metrics=['accuracy'],loss_weights=[1,mdd_weight,center_weight])
  return model


#Training and testing sets formulation

In [10]:
def mini_set(label_set,num):
  index1=np.where(label_set == 1)[0]
  index2=np.where(label_set == 2)[0]
  index3=np.where(label_set == 3)[0]
  index4=np.where(label_set == 4)[0]

  number=max(index1[num-1],index2[num-1],index3[num-1],index4[num-1])+1
  return number


In [11]:

def model_training(epochs,model,Subject_number,i):
  lsee=[]
  simple_model=model
  
  hyperparameter=10

  x_train,y_train,y_train_onehot,x_test,y_test,y_test_onehot = prepare_features('',Subject_number,False)
  selecty=np.hstack((y_train,y_test))

  train_mini_set=mini_set(selecty,hyperparameter)

  total_x=np.vstack((x_train,x_test))
  total_y=np.vstack((y_train_onehot,y_test_onehot))

  index1=np.where(selecty[:train_mini_set] == 1)[0]
  index2=np.where(selecty[:train_mini_set] == 2)[0]
  index3=np.where(selecty[:train_mini_set] == 3)[0]
  index4=np.where(selecty[:train_mini_set] == 4)[0]


  choice1=np.random.choice(index1,hyperparameter,replace=False)
  choice2=np.random.choice(index2,hyperparameter,replace=False)
  choice3=np.random.choice(index3,hyperparameter,replace=False)
  choice4=np.random.choice(index4,hyperparameter,replace=False)


  choice_total=np.hstack((choice1,choice2,choice3,choice4))


  test_list=list(range(train_mini_set))

  
  target_x=total_x[choice_total]
  target_y=total_y[choice_total]

  xxxtest= np.delete(total_x, test_list,0)
  yyytest=np.delete(total_y, test_list,0)




  total_x_train=[]
  total_y_train=[]
  full_subjectnum=[0,1,2,3,4,5,6,7,8]
  full_subjectnum.remove(Subject_number)
  for train_subject in full_subjectnum:
    x_train,y_train,y_train_onehot,x_test,y_test,y_test_onehot = prepare_features('',train_subject,False)

    if len(total_x_train)==0:
      total_x_train=x_train
      total_y_train=y_train_onehot
    else:
      total_x_train=np.vstack((total_x_train,x_train))
      total_y_train=np.vstack((total_y_train,y_train_onehot))
    total_x_train=np.vstack((total_x_train,x_test))
    total_y_train=np.vstack((total_y_train,y_test_onehot))
  x_train=total_x_train
  y_train=total_y_train

  train_xx=np.vstack((x_train,target_x))
  train_yy=np.vstack((y_train,target_y))


  test_ones=np.ones((len(target_x)))
  test_zeros=np.zeros((len(x_train)))
  test_domian_label=np.hstack((test_ones,test_zeros))



  x_test_target=xxxtest
  y_test_onehot_target=yyytest







  checkpoint = ModelCheckpoint('model_best_'+str(i)+'.h5', verbose=1, monitor='val_softmax_accuracy',save_best_only=True, mode='max')  


  valid_transform=np.ones((len(x_test_target)))
  dummy1 = np.zeros((train_xx.shape[0], 1))
  dummy2 = np.zeros((x_test_target.shape[0], 1))

  simple_model.fit([train_xx,train_yy],[train_yy,test_domian_label,dummy1],batch_size=128, epochs=epochs, callbacks=[checkpoint],
          verbose=0, validation_data=([x_test_target,y_test_onehot_target], [y_test_onehot_target,valid_transform,dummy2]))



In [12]:
epochs=500
mdd_weight=0.1
center_weight=0.01
Subject_number=2

model=model_complie(mdd_weight,center_weight)

model_training(epochs,model,Subject_number,i=2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/custom_loss/Reshape_25:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/custom_loss/Reshape_24:0", shape=(None, 120), dtype=float32), dense_shape=Tensor("gradient_tape/custom_loss/Cast_6:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/custom_loss/Reshape_27:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/custom_loss/Reshape_26:0", shape=(None, 120), dtype=float32), dense_shape=Tensor("gradient_tape/custom_loss/Cast_7:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. Th


Epoch 1: val_softmax_accuracy improved from -inf to 0.40600, saving model to model_best_2.h5

Epoch 2: val_softmax_accuracy improved from 0.40600 to 0.41400, saving model to model_best_2.h5

Epoch 3: val_softmax_accuracy did not improve from 0.41400

Epoch 4: val_softmax_accuracy did not improve from 0.41400

Epoch 5: val_softmax_accuracy improved from 0.41400 to 0.46400, saving model to model_best_2.h5

Epoch 6: val_softmax_accuracy improved from 0.46400 to 0.49400, saving model to model_best_2.h5

Epoch 7: val_softmax_accuracy did not improve from 0.49400

Epoch 8: val_softmax_accuracy improved from 0.49400 to 0.61000, saving model to model_best_2.h5

Epoch 9: val_softmax_accuracy did not improve from 0.61000

Epoch 10: val_softmax_accuracy did not improve from 0.61000

Epoch 11: val_softmax_accuracy improved from 0.61000 to 0.62200, saving model to model_best_2.h5

Epoch 12: val_softmax_accuracy improved from 0.62200 to 0.63600, saving model to model_best_2.h5

Epoch 13: val_softma